In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

# from moseq2_nlp.data import get_embedding_representation, get_transition_representation, get_usage_representation, load_groups
import moseq2_nlp.train as trainer
from typing import Literal

from moseq2_viz.util import parse_index, read_yaml
from moseq2_viz.model.util import (get_transition_matrix, parse_model_results)
from moseq2_nlp.data import get_embedding_representation, load_groups, get_raw_data
from moseq2_nlp.utils import ensure_dir, write_yaml
from moseq2_nlp.train import train_regressor
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
!pip install lime

In [ ]:
custom_groupings = []
model_path = '/Users/aishwaryaharpale/Downloads/Abraira-Lab/2021-02-19_Meloxicam/rOT_model_1000.p'
index_path = '/Users/aishwaryaharpale/Downloads/Abraira-Lab/2021-02-19_Meloxicam/moseq2-index.role.yaml'
group_map = load_groups(index_path, custom_groupings)

In [ ]:
train_config = '/Users/aishwaryaharpale/tischfield-lab/moseq2-nlp/train-config.yaml'
config_data = read_yaml(train_config)


In [ ]:
# for key, val in config_data.items():
#     print(key, " : ", val, " \n")

exp_dir = ensure_dir(os.path.join(config_data['save_dir'], config_data['name']))

In [ ]:
labels, features = get_embedding_representation(config_data['model_path'], config_data['index_path'], group_map, emissions=True, bad_syllables=config_data['bad_syllables'],
                            dm=config_data['dm'], embedding_dim=config_data['embedding_dim'], embedding_window=config_data['embedding_window'], embedding_epochs=config_data['embedding_epochs'], 
                            min_count=config_data['min_count'],
                            model_dest=os.path.join(exp_dir, 'doc2vec'), ablation='none', phrase_path=None, seed=config_data['seed'])

In [ ]:
# This is a DBOW based model only (Does not used DM-Distributed Memory)
model = Doc2Vec.load("/Users/aishwaryaharpale/tischfield-lab/Reverse-Mapping/rOT/doc2vec.model")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=config_data['test_size'], random_state=config_data['split_seed'], stratify=labels)
clf = train_regressor(X_train, y_train, config_data['k'], config_data['penalty'], config_data['num_c'], config_data['seed'], config_data['multi_class'])
class_names = list(set(labels))

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names, bow=True)

In [ ]:
sentences, out_groups = get_raw_data(config_data['model_path'], config_data['index_path'], max_syllable=100, \
                            emissions=config_data['emissions'], bad_syllables=config_data['bad_syllables'], ablation='none', phrase_path=None)

In [ ]:
def new_predict(sentences):
    embeddings = []
    for sent in sentences:
        embeddings.append(model.infer_vector(sent.split(' ')))
    embeddings = np.array(embeddings)
    return clf.predict_proba(embeddings.reshape(-1, 70))

In [ ]:
embedding = model.infer_vector(sentences[0])
# embedding_string = []
# for emb in embedding:
#     embedding_string.append(str(emb))
print(clf.predict_proba(embedding.reshape(-1,70)))
#     embedding_string.append(str(emb))
print(embedding.reshape(-1,70).shape)
print(embedding.shape)

In [ ]:
sent1 = ' '.join(item for item in sentences[0])
print("Actual label : ",out_groups[0])
exp1 = explainer.explain_instance(sent1,new_predict, num_samples=80)
fig = exp1.as_pyplot_figure()

In [ ]:
sent2 = ' '.join(item for item in sentences[1])
print("Actual label : ",out_groups[1])
exp2 = explainer.explain_instance(sent2,new_predict, num_samples=80)
fig = exp2.as_pyplot_figure()